In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GROQ_API_KEY')

# Verificar que la key se cargó correctamente
print("API Key loaded:", bool(api_key))

API Key loaded: True


In [2]:
from groq import Groq

# Inicializar el cliente de Groq usando la API key que ya cargamos
client = Groq(api_key=api_key)

# Crear una completion usando el chat
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "¿Cuáles son las principales características del modelo llama 3.3?"
        }
    ],
    model="llama-3.3-70b-versatile",  # Usando el modelo llama 3.3
    temperature=0.7,  # Ajusta la creatividad de las respuestas (0-1)
    max_tokens=1024  # Máximo de tokens en la respuesta
)

# Imprimir la respuesta
print(chat_completion.choices[0].message.content)

El modelo Llama 3.3 es una versión avanzada del modelo de lenguaje grande desarrollado por Meta, diseñado para procesar y generar texto similar a como lo haría un humano. A continuación, se presentan algunas de las principales características del modelo Llama 3.3:

1. **Capacidad de procesamiento de lenguaje**: El modelo Llama 3.3 cuenta con una capacidad de procesamiento de lenguaje avanzada, lo que le permite entender y generar texto en una variedad de estilos y formatos, desde conversaciones informales hasta textos formales y técnicos.

2. **Aprendizaje automático**: El modelo se entrenó utilizando técnicas de aprendizaje automático, lo que le permite aprender de grandes cantidades de datos y mejorar su rendimiento con el tiempo.

3. **Conocimiento general**: El modelo Llama 3.3 tiene acceso a una gran base de conocimientos generales, lo que le permite proporcionar respuestas precisas y relevantes a una amplia variedad de preguntas y temas.

4. **Generación de texto**: El modelo pue

In [3]:
def ask_llama(prompt: str, temperature: float = 0.7) -> str:
    """
    Helper function to make queries to llama 3.3
    """
    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.3-70b-versatile",
        temperature=temperature,
        max_tokens=1024
    )
    return completion.choices[0].message.content

# Ejemplo de uso
response = ask_llama("Explica el concepto de inteligencia artificial en términos simples")
print(response)

**Introducción a la Inteligencia Artificial**

La inteligencia artificial (IA) es un campo de la informática que se enfoca en crear máquinas y sistemas que puedan realizar tareas que normalmente requieren la inteligencia humana. En otras palabras, la IA busca crear sistemas que puedan pensar, aprender y actuar como los seres humanos.

**¿Qué es la Inteligencia Artificial?**

La IA se puede definir como la capacidad de una máquina o sistema para:

1. **Aprender**: La IA puede aprender de los datos y experiencias para mejorar su rendimiento en una tarea específica.
2. **Razonar**: La IA puede analizar información y tomar decisiones basadas en ella.
3. **Percebir**: La IA puede interpretar y comprender el entorno que la rodea.

**Tipos de Inteligencia Artificial**

Existen dos tipos principales de IA:

1. **IA débil**: Se enfoca en realizar tareas específicas, como el reconocimiento de voz o la clasificación de imágenes.
2. **IA fuerte**: Se enfoca en crear sistemas que puedan pensar y ra

In [7]:
class TriageAgent:
    def __init__(self, client):
        self.client = client
        self.system_prompt = """
        Eres un agente de triaje médico experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "urgency_level": (número del 1 al 5),
            "department": "nombre del departamento",
            "immediate_actions": ["acción 1", "acción 2", ...],
            "reasoning": "explicación de la decisión"
        }

        Niveles de urgencia:
        1 - Resucitación (atención inmediata)
        2 - Emergencia (muy urgente, minutos)
        3 - Urgente (hasta 1 hora)
        4 - Menos urgente (hasta 2 horas)
        5 - No urgente (hasta 4 horas)
        """

    def evaluate_patient(self, symptoms: str) -> Dict:
        prompt = f"""
        Evalúa los siguientes síntomas y responde SOLO con un JSON válido:
        {symptoms}
        """
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        try:
            response = self.client.chat.completions.create(
                messages=messages,
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}  # Forzamos respuesta en JSON
            )
            
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError as e:
            return {
                "error": "Error al procesar JSON",
                "details": str(e),
                "raw_response": response.choices[0].message.content
            }
        except Exception as e:
            return {
                "error": "Error general",
                "details": str(e)
            }

In [8]:

# Probemos el agente actualizado
triage_agent = TriageAgent(client)

In [9]:
# Caso de prueba 1: Emergencia cardíaca
symptoms_1 = "Paciente de 45 años con dolor en el pecho, dificultad para respirar y sudoración excesiva desde hace 30 minutos"
evaluation_1 = triage_agent.evaluate_patient(symptoms_1)
print("Caso 1 - Posible emergencia cardíaca:")
print(json.dumps(evaluation_1, indent=2, ensure_ascii=False))

Caso 1 - Posible emergencia cardíaca:
{
  "urgency_level": 1,
  "department": "Cardiología",
  "immediate_actions": [
    "Llamar a emergencias",
    "Administrar oxígeno",
    "Realizar un electrocardiograma (ECG)",
    "Administrar aspirina si está indicada"
  ],
  "reasoning": "Los síntomas de dolor en el pecho, dificultad para respirar y sudoración excesiva sugieren un posible infarto de miocardio, lo que requiere atención inmediata para prevenir daños cardiacos irreversibles."
}


In [10]:
# Caso de prueba 2: Caso menos urgente
symptoms_2 = "Paciente de 25 años con dolor de garganta y fiebre baja (37.5°C) desde hace 2 días"
evaluation_2 = triage_agent.evaluate_patient(symptoms_2)
print("\nCaso 2 - Síntomas leves:")
print(json.dumps(evaluation_2, indent=2, ensure_ascii=False))


Caso 2 - Síntomas leves:
{
  "urgency_level": 4,
  "department": "Medicina General",
  "immediate_actions": [
    "Realizar un examen físico",
    "Tomar muestras para cultivo de garganta si es necesario",
    "Prescribir medicación para el dolor y la fiebre"
  ],
  "reasoning": "El paciente presenta síntomas de una infección respiratoria leve, como dolor de garganta y fiebre baja, que pueden ser manejados con tratamiento sintomático y seguimiento médico sin requerir atención inmediata."
}


In [11]:
class SpecialistAgent:
    def __init__(self, client):
        self.client = client
        self.system_prompt = """
        Eres un médico especialista experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "diagnosis": "diagnóstico preliminar",
            "confidence_level": (número del 1 al 100),
            "required_tests": ["test 1", "test 2", ...],
            "treatment_plan": ["paso 1", "paso 2", ...],
            "prescriptions": ["medicamento 1", "medicamento 2", ...],
            "follow_up": "tiempo recomendado para siguiente revisión",
            "warnings": ["advertencia 1", "advertencia 2", ...],
            "specialist_notes": "notas adicionales importantes"
        }

        Asegúrate de:
        1. Proporcionar un diagnóstico basado en la evidencia
        2. Sugerir pruebas relevantes
        3. Crear un plan de tratamiento específico
        4. Incluir advertencias y contraindicaciones importantes
        """

    def evaluate_case(self, patient_data: str, specialty: str) -> Dict:
        prompt = f"""
        Como especialista en {specialty}, evalúa el siguiente caso:
        {patient_data}
        
        Proporciona una evaluación detallada y plan de tratamiento.
        """
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt}
        ]
        
        try:
            response = self.client.chat.completions.create(
                messages=messages,
                model="llama-3.3-70b-versatile",
                temperature=0.3,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            return {
                "error": "Error en la evaluación",
                "details": str(e)
            }

In [12]:
# Crear instancia del especialista

import json

specialist = SpecialistAgent(client)

# Probar con un caso cardíaco
cardiac_case = """
Paciente masculino de 45 años con:
- Dolor torácico agudo
- Presión arterial: 160/95
- Frecuencia cardíaca: 95 bpm
- ECG muestra elevación del segmento ST
- Antecedentes de hipertensión
- Fumador activo
"""

evaluation = specialist.evaluate_case(cardiac_case, "Cardiología")
print(json.dumps(evaluation, indent=2, ensure_ascii=False))

{
  "diagnosis": "Infarto de miocardio agudo",
  "confidence_level": 90,
  "required_tests": [
    "Troponina",
    "CK-MB",
    "ECG de seguimiento",
    "Ecocardiograma",
    "Angiografía coronaria"
  ],
  "treatment_plan": [
    "Administrar aspirina 325 mg vía oral de inmediato",
    "Nitroglicerina sublingual para aliviar el dolor torácico",
    "Beta-bloqueantes para controlar la frecuencia cardíaca y la presión arterial",
    "Anticoagulación con heparina",
    "Considerar trombolisis o intervención coronaria percutánea (ICP) según la disponibilidad y el estado del paciente"
  ],
  "prescriptions": [
    "Aspirina 81 mg vía oral cada 24 horas",
    "Metoprolol 25 mg vía oral cada 12 horas",
    "Atorvastatina 80 mg vía oral cada 24 horas",
    "Nitroglicerina sublingual según sea necesario"
  ],
  "follow_up": "Revisión en 1 semana para evaluar la respuesta al tratamiento y ajustar según sea necesario",
  "warnings": [
    "Riesgo de complicaciones como insuficiencia cardíaca, a

In [13]:
def process_emergency_case(triage_agent, specialist_agent, symptoms: str):
    """
    Procesa un caso de emergencia usando ambos agentes
    """
    # Primero hacemos el triaje
    triage_result = triage_agent.evaluate_patient(symptoms)
    
    # Si es urgente (nivel 1 o 2), consultamos inmediatamente con el especialista
    if triage_result.get("urgency_level", 5) <= 2:
        department = triage_result["department"]
        specialist_evaluation = specialist_agent.evaluate_case(symptoms, department)
        
        return {
            "triage": triage_result,
            "specialist_evaluation": specialist_evaluation,
            "status": "URGENT_CARE_REQUIRED"
        }
    
    return {
        "triage": triage_result,
        "status": "REGULAR_CARE"
    }

In [14]:
# Probar el sistema integrado
case = "Paciente de 58 años con dolor intenso en el pecho, irradiando al brazo izquierdo, sudoración fría y náuseas desde hace 45 minutos"
result = process_emergency_case(triage_agent, specialist, case)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "triage": {
    "urgency_level": 1,
    "department": "Cardiología",
    "immediate_actions": [
      "Llamar a emergencias",
      "Administrar oxígeno",
      "Monitorear signos vitales",
      "Preparar para posible reanimación cardiopulmonar"
    ],
    "reasoning": "Los síntomas descritos sugieren un posible infarto de miocardio, que requiere atención médica inmediata para prevenir daños cardiacos irreversibles y reducir el riesgo de muerte."
  },
  "specialist_evaluation": {
    "diagnosis": "Infarto de miocardio agudo (posible)",
    "confidence_level": 90,
    "required_tests": [
      "Electrocardiograma (ECG) de 12 derivaciones",
      "Análisis de sangre para enzimas cardíacas (troponina, CK-MB)",
      "Ecocardiograma transtorácico",
      "Angiografía coronaria"
    ],
    "treatment_plan": [
      "Administrar aspirina 325 mg vía oral de inmediato",
      "Oxígeno suplementario si la saturación de oxígeno es < 90%",
      "Nitroglicerina sublingual para aliviar el dol

In [19]:
class NurseAgent:
    def __init__(self, client):
        self.client = client
        self.patient_history = {}  # Para almacenar el historial de cada paciente
        self.system_prompt = """
        Eres un enfermero/a experto/a. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "vital_signs": {
                "temperature": "valor en °C",
                "blood_pressure": "sistólica/diastólica",
                "heart_rate": "latidos por minuto",
                "respiratory_rate": "respiraciones por minuto",
                "oxygen_saturation": "porcentaje",
                "pain_level": "escala 0-10"
            },
            "patient_status": "estado actual del paciente",
            "care_tasks": ["tarea 1", "tarea 2", ...],
            "medication_administration": [
                {
                    "medication": "nombre del medicamento",
                    "dose": "dosis",
                    "route": "vía de administración",
                    "time": "hora de administración",
                    "status": "administrado/pendiente/retrasado"
                }
            ],
            "observations": ["observación 1", "observación 2", ...],
            "alerts": ["alerta 1", "alerta 2", ...],
            "next_check": "tiempo para próxima revisión",
            "shift_notes": "notas importantes para el siguiente turno",
            "patient_education": ["tema educativo 1", "tema educativo 2", ...],
            "care_plan_progress": {
                "goals_met": ["objetivo 1", "objetivo 2", ...],
                "pending_goals": ["objetivo pendiente 1", "objetivo pendiente 2", ...],
                "complications": ["complicación 1", "complicación 2", ...]
            }
        }
        """

    def monitor_patient(self, patient_id: str, patient_data: str, care_plan: Dict) -> Dict:
        """
        Monitorea al paciente y actualiza su historial
        """
        current_assessment = self._get_assessment(patient_data, care_plan)
        self._update_patient_history(patient_id, current_assessment)
        return current_assessment

    def _get_assessment(self, patient_data: str, care_plan: Dict) -> Dict:
        """
        Obtiene la evaluación actual del paciente
        """
        prompt = f"""
        Evalúa y monitorea al siguiente paciente:
        
        Datos del paciente:
        {patient_data}
        
        Plan de cuidados prescrito:
        {json.dumps(care_plan, indent=2)}
        
        Proporciona una evaluación de enfermería detallada y plan de cuidados.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            return {
                "error": "Error en la evaluación de enfermería",
                "details": str(e)
            }

    def _update_patient_history(self, patient_id: str, assessment: Dict) -> None:
        """
        Actualiza el historial del paciente
        """
        if patient_id not in self.patient_history:
            self.patient_history[patient_id] = []
        self.patient_history[patient_id].append({
            "timestamp": datetime.now().isoformat(),
            "assessment": assessment
        })

    def get_patient_history(self, patient_id: str) -> List[Dict]:
        """
        Obtiene el historial completo del paciente
        """
        return self.patient_history.get(patient_id, [])

    def check_vital_signs_trends(self, patient_id: str) -> Dict:
        """
        Analiza tendencias en los signos vitales
        """
        history = self.get_patient_history(patient_id)
        if not history:
            return {"error": "No hay historial disponible"}

        vital_signs_history = []
        for entry in history:
            if "vital_signs" in entry["assessment"]:
                vital_signs_history.append({
                    "timestamp": entry["timestamp"],
                    "vital_signs": entry["assessment"]["vital_signs"]
                })

        return {
            "vital_signs_history": vital_signs_history,
            "trends": self._analyze_trends(vital_signs_history)
        }

    def _analyze_trends(self, vital_signs_history: List[Dict]) -> Dict:
        """
        Analiza tendencias en los signos vitales y genera alertas si es necesario
        """
        if not vital_signs_history:
            return {"message": "No hay suficientes datos para analizar tendencias"}

        # Aquí iría la lógica de análisis de tendencias
        return {
            "improving": [],
            "stable": [],
            "worsening": [],
            "alerts": []
        }

In [20]:

# Probemos las nuevas funcionalidades
nurse = NurseAgent(client)

In [21]:

# Importamos datetime para el registro temporal
from datetime import datetime

# Caso de prueba
patient_id = "P001"
test_case = """
Paciente de 58 años con dolor torácico.
Diagnóstico: Infarto agudo de miocardio
Estado actual: Consciente, orientado, con dolor controlado
"""

# Primera evaluación
initial_assessment = nurse.monitor_patient(patient_id, test_case, care_plan)
print("Evaluación inicial:")
print(json.dumps(initial_assessment, indent=2, ensure_ascii=False))

Evaluación inicial:
{
  "vital_signs": {
    "temperature": "36.5 °C",
    "blood_pressure": "120/80 mmHg",
    "heart_rate": "80 latidos por minuto",
    "respiratory_rate": "18 respiraciones por minuto",
    "oxygen_saturation": "95%",
    "pain_level": "2/10"
  },
  "patient_status": "Estable, con dolor controlado",
  "care_tasks": [
    "Monitoreo continuo de los signos vitales",
    "Administración de medicamentos según el plan de tratamiento",
    "Evaluación del dolor y ajuste de la medicación según sea necesario",
    "Educación al paciente sobre la importancia de la adherencia al tratamiento y el seguimiento médico",
    "Apoyo emocional y psicológico al paciente y su familia"
  ],
  "medication_administration": [
    {
      "medication": "Aspirina 325 mg",
      "dose": "1 tableta",
      "route": "vía oral",
      "time": "inmediato",
      "status": "administrado"
    },
    {
      "medication": "Nitroglicerina sublingual",
      "dose": "1 tableta",
      "route": "subli

In [22]:

# Simulamos una segunda evaluación después de un tiempo
test_case_followup = """
Paciente de 58 años en seguimiento por IAM.
Estado actual: Mejora del dolor, signos vitales estables
"""

followup_assessment = nurse.monitor_patient(patient_id, test_case_followup, care_plan)
print("\nSeguimiento:")
print(json.dumps(followup_assessment, indent=2, ensure_ascii=False))


Seguimiento:
{
  "vital_signs": {
    "temperature": "36.5 °C",
    "blood_pressure": "120/80 mmHg",
    "heart_rate": "72 latidos por minuto",
    "respiratory_rate": "16 respiraciones por minuto",
    "oxygen_saturation": "98%",
    "pain_level": "2/10"
  },
  "patient_status": "Estable, con mejora del dolor torácico",
  "care_tasks": [
    "Monitoreo continuo de signos vitales",
    "Administración de medicamentos según prescripción",
    "Educación al paciente sobre el manejo del dolor y la importancia de la adherencia al tratamiento",
    "Apoyo emocional y psicológico",
    "Preparación para pruebas diagnósticas adicionales"
  ],
  "medication_administration": [
    {
      "medication": "Aspirina",
      "dose": "81 mg",
      "route": "vía oral",
      "time": "08:00 horas",
      "status": "administrado"
    },
    {
      "medication": "Metoprolol",
      "dose": "25 mg",
      "route": "vía oral",
      "time": "08:00 horas",
      "status": "administrado"
    },
    {
    

In [23]:
# Verificamos el historial y tendencias
history = nurse.check_vital_signs_trends(patient_id)
print("\nTendencias de signos vitales:")
print(json.dumps(history, indent=2, ensure_ascii=False))


Tendencias de signos vitales:
{
  "vital_signs_history": [
    {
      "timestamp": "2025-01-11T23:45:12.635314",
      "vital_signs": {
        "temperature": "36.5 °C",
        "blood_pressure": "120/80 mmHg",
        "heart_rate": "80 latidos por minuto",
        "respiratory_rate": "18 respiraciones por minuto",
        "oxygen_saturation": "95%",
        "pain_level": "2/10"
      }
    },
    {
      "timestamp": "2025-01-11T23:45:35.401081",
      "vital_signs": {
        "temperature": "36.5 °C",
        "blood_pressure": "120/80 mmHg",
        "heart_rate": "72 latidos por minuto",
        "respiratory_rate": "16 respiraciones por minuto",
        "oxygen_saturation": "98%",
        "pain_level": "2/10"
      }
    }
  ],
  "trends": {
    "improving": [],
    "stable": [],
    "worsening": [],
    "alerts": []
  }
}


In [26]:
def process_hospital_case(triage_agent, specialist_agent, nurse_agent, patient_case: str):
    """
    Procesa un caso hospitalario completo usando los tres agentes
    """
    # Generamos un ID único para el paciente
    patient_id = f"P{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    # 1. Triaje inicial
    triage_result = triage_agent.evaluate_patient(patient_case)
    
    # 2. Si es urgente, evaluación del especialista
    if triage_result.get("urgency_level", 5) <= 3:
        department = triage_result["department"]
        specialist_evaluation = specialist_agent.evaluate_case(patient_case, department)
        
        # 3. Plan de cuidados de enfermería
        nurse_care = nurse_agent.monitor_patient(
            patient_id=patient_id,
            patient_data=patient_case,
            care_plan=specialist_evaluation
        )
        
        return {
            "case_status": "ACTIVE",
            "patient_id": patient_id,
            "triage_evaluation": triage_result,
            "specialist_evaluation": specialist_evaluation,
            "nursing_care": nurse_care,
            "next_steps": [
                action for action in triage_result.get("immediate_actions", [])
            ] + [
                f"Administrar {med}" for med in specialist_evaluation.get("prescriptions", [])
            ]
        }
    
    return {
        "case_status": "REGULAR_CARE",
        "patient_id": patient_id,
        "triage_evaluation": triage_result,
        "next_steps": triage_result.get("immediate_actions", [])
    }

In [27]:
# Probemos el sistema completo
test_case = """
Paciente de 58 años con:
- Dolor intenso en el pecho que irradia al brazo izquierdo
- Sudoración fría
- Náuseas
- Antecedentes de hipertensión
- Fumador de 20 cigarrillos/día
Tiempo de evolución: 45 minutos
"""

complete_evaluation = process_hospital_case(triage_agent, specialist, nurse, test_case)
print(json.dumps(complete_evaluation, indent=2, ensure_ascii=False))

{
  "case_status": "ACTIVE",
  "patient_id": "P20250111234654",
  "triage_evaluation": {
    "urgency_level": 1,
    "department": "Cardiología",
    "immediate_actions": [
      "Llamar a emergencias",
      "Administrar oxígeno",
      "Monitorear ritmo cardíaco",
      "Administrar aspirina si está indicada"
    ],
    "reasoning": "Los síntomas del paciente, como dolor intenso en el pecho que irradia al brazo izquierdo, sudoración fría y náuseas, sugieren un posible infarto de miocardio, especialmente considerando su historial de hipertensión y tabaquismo. La evolución rápida de los síntomas en 45 minutos aumenta la urgencia de la situación, requiriendo atención inmediata."
  },
  "specialist_evaluation": {
    "diagnosis": "Infarto de miocardio agudo (posible IAM con elevación del segmento ST)",
    "confidence_level": 90,
    "required_tests": [
      "Electrocardiograma (ECG) de 12 derivaciones",
      "Troponina sérica",
      "Prueba de estrés con ecocardiograma o prueba de es

In [28]:
class PharmacistAgent:
    def __init__(self, client):
        self.client = client
        self.medication_history = {}  # Historial de medicamentos por paciente
        self.system_prompt = """
        Eres un farmacéutico experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "prescription_review": {
                "is_approved": boolean,
                "warnings": ["advertencia 1", "advertencia 2", ...],
                "interactions": ["interacción 1", "interacción 2", ...],
                "dosage_check": {
                    "is_correct": boolean,
                    "suggestions": ["sugerencia 1", "sugerencia 2", ...]
                }
            },
            "alternatives": [
                {
                    "original_med": "medicamento original",
                    "alternatives": ["alternativa 1", "alternativa 2", ...],
                    "reason": "razón de la sugerencia"
                }
            ],
            "administration_instructions": [
                {
                    "medication": "nombre del medicamento",
                    "instructions": ["instrucción 1", "instrucción 2", ...],
                    "special_considerations": ["consideración 1", "consideración 2", ...]
                }
            ],
            "inventory_status": {
                "available": ["medicamento 1", "medicamento 2", ...],
                "low_stock": ["medicamento 1", "medicamento 2", ...],
                "out_of_stock": ["medicamento 1", "medicamento 2", ...]
            }
        }
        """

    def review_prescription(self, patient_id: str, prescriptions: List[str], patient_data: str) -> Dict:
        prompt = f"""
        Revisa las siguientes prescripciones para el paciente:
        
        Datos del paciente:
        {patient_data}
        
        Prescripciones:
        {json.dumps(prescriptions, indent=2)}
        
        Realiza una evaluación farmacéutica completa.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            review = json.loads(response.choices[0].message.content)
            self._update_medication_history(patient_id, prescriptions, review)
            return review
        except Exception as e:
            return {
                "error": "Error en la revisión farmacéutica",
                "details": str(e)
            }

    def _update_medication_history(self, patient_id: str, prescriptions: List[str], review: Dict) -> None:
        if patient_id not in self.medication_history:
            self.medication_history[patient_id] = []
        
        self.medication_history[patient_id].append({
            "timestamp": datetime.now().isoformat(),
            "prescriptions": prescriptions,
            "review": review
        })

    def get_medication_history(self, patient_id: str) -> List[Dict]:
        return self.medication_history.get(patient_id, [])

In [29]:
# Probemos el PharmacistAgent
pharmacist = PharmacistAgent(client)

# Caso de prueba con las prescripciones del caso cardíaco
test_prescriptions = [
    "Aspirina 81-100 mg diarios",
    "Metoprolol 25-50 mg dos veces al día",
    "Atorvastatina 80 mg diarios",
    "Nitroglicerina sublingual PRN"
]

test_patient_data = """
Paciente de 58 años con infarto agudo de miocardio
Antecedentes:
- Hipertensión
- Fumador activo
- Sin alergias conocidas
"""

pharmacy_review = pharmacist.review_prescription("P001", test_prescriptions, test_patient_data)
print("Revisión farmacéutica:")
print(json.dumps(pharmacy_review, indent=2, ensure_ascii=False))

Revisión farmacéutica:
{
  "prescription_review": {
    "is_approved": true,
    "warnings": [
      "El paciente debe ser monitoreado estrechamente debido a su historial de infarto agudo de miocardio y hipertensión",
      "Se debe considerar la interacción entre Metoprolol y otros medicamentos que puedan afectar la presión arterial",
      "La dosis de Atorvastatina es alta, se debe monitorear los niveles de lípidos en sangre y la función hepática"
    ],
    "interactions": [
      "Interacción entre Aspirina y Metoprolol: puede aumentar el riesgo de sangrado",
      "Interacción entre Atorvastatina y otros medicamentos que afectan el metabolismo hepático"
    ],
    "dosage_check": {
      "is_correct": true,
      "suggestions": [
        "Considerar la reducción de la dosis de Metoprolol si el paciente presenta efectos secundarios como bradicardia",
        "Ajustar la dosis de Atorvastatina según los niveles de lípidos en sangre y la respuesta del paciente"
      ]
    }
  },
  

In [30]:
class LaboratoryAgent:
    def __init__(self, client):
        self.client = client
        self.lab_results = {}  # Historial de resultados por paciente
        self.system_prompt = """
        Eres un especialista de laboratorio clínico. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "test_interpretation": {
                "results": [
                    {
                        "test_name": "nombre del test",
                        "value": "valor numérico o cualitativo",
                        "reference_range": "rango de referencia",
                        "interpretation": "interpretación del resultado",
                        "urgency_level": "normal/urgente/crítico"
                    }
                ],
                "critical_values": ["valor crítico 1", "valor crítico 2", ...],
                "recommendations": ["recomendación 1", "recomendación 2", ...]
            },
            "additional_tests": [
                {
                    "test": "nombre del test adicional",
                    "reason": "razón de la recomendación",
                    "priority": "alta/media/baja"
                }
            ],
            "trending_analysis": {
                "improving": ["parámetro 1", "parámetro 2", ...],
                "stable": ["parámetro 1", "parámetro 2", ...],
                "worsening": ["parámetro 1", "parámetro 2", ...]
            }
        }
        """

    def process_results(self, patient_id: str, test_results: Dict, clinical_context: str) -> Dict:
        prompt = f"""
        Interpreta los siguientes resultados de laboratorio:
        
        Contexto clínico:
        {clinical_context}
        
        Resultados:
        {json.dumps(test_results, indent=2)}
        
        Proporciona una interpretación completa y recomendaciones.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            interpretation = json.loads(response.choices[0].message.content)
            self._update_lab_history(patient_id, test_results, interpretation)
            return interpretation
        except Exception as e:
            return {
                "error": "Error en el procesamiento de resultados",
                "details": str(e)
            }

    def _update_lab_history(self, patient_id: str, results: Dict, interpretation: Dict) -> None:
        if patient_id not in self.lab_results:
            self.lab_results[patient_id] = []
        
        self.lab_results[patient_id].append({
            "timestamp": datetime.now().isoformat(),
            "results": results,
            "interpretation": interpretation
        })

In [31]:
# Probemos el LaboratoryAgent
lab = LaboratoryAgent(client)

In [32]:

# Caso de prueba con resultados de laboratorio cardíacos
test_results = {
    "cardiac_markers": {
        "troponin_i": "2.5 ng/mL",
        "ck_mb": "25 U/L",
        "myoglobin": "150 ng/mL"
    },
    "blood_count": {
        "hemoglobin": "14.5 g/dL",
        "wbc": "9.5 x10^9/L",
        "platelets": "250 x10^9/L"
    },
    "chemistry": {
        "sodium": "138 mEq/L",
        "potassium": "4.2 mEq/L",
        "creatinine": "1.1 mg/dL"
    }
}

In [33]:
clinical_context = """
Paciente con sospecha de infarto agudo de miocardio
Síntomas: Dolor torácico, diaforesis, náuseas
Tiempo de evolución: 45 minutos
"""

lab_interpretation = lab.process_results("P001", test_results, clinical_context)
print("Interpretación de laboratorio:")
print(json.dumps(lab_interpretation, indent=2, ensure_ascii=False))

Interpretación de laboratorio:
{
  "test_interpretation": {
    "results": [
      {
        "test_name": "Troponina I",
        "value": "2.5 ng/mL",
        "reference_range": "< 0.04 ng/mL",
        "interpretation": "Elevada, sugiere daño miocárdico",
        "urgency_level": "crítico"
      },
      {
        "test_name": "CK-MB",
        "value": "25 U/L",
        "reference_range": "0-24 U/L",
        "interpretation": "Ligeramente elevada, puede indicar daño miocárdico",
        "urgency_level": "urgente"
      },
      {
        "test_name": "Mioglobina",
        "value": "150 ng/mL",
        "reference_range": "0-100 ng/mL",
        "interpretation": "Elevada, sugiere daño muscular",
        "urgency_level": "urgente"
      },
      {
        "test_name": "Hemoglobina",
        "value": "14.5 g/dL",
        "reference_range": "13.5-17.5 g/dL",
        "interpretation": "Normal",
        "urgency_level": "normal"
      },
      {
        "test_name": "Leucocitos",
        "val

In [34]:
class RadiologyAgent:
    def __init__(self, client):
        self.client = client
        self.imaging_history = {}  # Historial de estudios por paciente
        self.system_prompt = """
        Eres un radiólogo experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "study_interpretation": {
                "study_type": "tipo de estudio",
                "findings": [
                    {
                        "location": "ubicación anatómica",
                        "description": "descripción del hallazgo",
                        "severity": "normal/leve/moderado/severo",
                        "urgency_level": "rutina/urgente/crítico"
                    }
                ],
                "comparison": "comparación con estudios previos si existen",
                "impression": "impresión diagnóstica general"
            },
            "recommendations": [
                {
                    "action": "acción recomendada",
                    "priority": "alta/media/baja",
                    "timeframe": "tiempo recomendado"
                }
            ],
            "technical_details": {
                "image_quality": "calidad de la imagen",
                "limitations": ["limitación 1", "limitación 2", ...],
                "radiation_dose": "dosis de radiación si aplica",
                "contrast_used": "tipo de contraste si se utilizó"
            },
            "follow_up": {
                "required": boolean,
                "timeframe": "tiempo recomendado",
                "study_type": "tipo de estudio recomendado",
                "reason": "razón del seguimiento"
            }
        }
        """

    def interpret_study(self, patient_id: str, study_data: Dict, clinical_context: str) -> Dict:
        prompt = f"""
        Interpreta el siguiente estudio radiológico:
        
        Contexto clínico:
        {clinical_context}
        
        Datos del estudio:
        {json.dumps(study_data, indent=2)}
        
        Proporciona una interpretación radiológica completa.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            interpretation = json.loads(response.choices[0].message.content)
            self._update_imaging_history(patient_id, study_data, interpretation)
            return interpretation
        except Exception as e:
            return {
                "error": "Error en la interpretación radiológica",
                "details": str(e)
            }

    def _update_imaging_history(self, patient_id: str, study_data: Dict, interpretation: Dict) -> None:
        if patient_id not in self.imaging_history:
            self.imaging_history[patient_id] = []
        
        self.imaging_history[patient_id].append({
            "timestamp": datetime.now().isoformat(),
            "study_data": study_data,
            "interpretation": interpretation
        })

    def get_imaging_history(self, patient_id: str) -> List[Dict]:
        return self.imaging_history.get(patient_id, [])

In [35]:
# Probemos el RadiologyAgent
radiologist = RadiologyAgent(client)

In [36]:
# Caso de prueba con una radiografía de tórax
chest_xray_data = {
    "study_type": "Radiografía de tórax PA y lateral",
    "acquisition_date": datetime.now().isoformat(),
    "technical_parameters": {
        "kV": "120",
        "mAs": "2.5",
        "image_format": "DICOM",
        "views": ["PA", "Lateral"]
    },
    "patient_position": "De pie",
    "previous_studies": False
}

clinical_context = """
Paciente de 58 años con sospecha de infarto agudo de miocardio.
Síntomas:
- Dolor torácico que irradia al brazo izquierdo
- Dificultad respiratoria
- Sudoración fría
Antecedentes:
- Hipertensión
- Fumador activo
"""

radiology_interpretation = radiologist.interpret_study("P001", chest_xray_data, clinical_context)
print("Interpretación radiológica:")
print(json.dumps(radiology_interpretation, indent=2, ensure_ascii=False))

Interpretación radiológica:
{
  "study_interpretation": {
    "study_type": "Radiografía de tórax PA y lateral",
    "findings": [
      {
        "location": "Cardiovasculares",
        "description": "Aumento del tamaño cardíaco con signos de congestión pulmonar bilateral leve",
        "severity": "Moderado",
        "urgency_level": "Urgente"
      },
      {
        "location": "Pulmones",
        "description": "Infiltrados intersticiales bilaterales sugestivos de edema pulmonar",
        "severity": "Moderado",
        "urgency_level": "Urgente"
      }
    ],
    "comparison": "No hay estudios previos para comparar",
    "impression": "Posible infarto agudo de miocardio con signos de insuficiencia cardíaca congestiva"
  },
  "recommendations": [
    {
      "action": "Realizar un ecocardiograma para evaluar la función cardíaca",
      "priority": "Alta",
      "timeframe": "Inmediato"
    },
    {
      "action": "Realizar un estudio de troponina para confirmar el diagnóstico d

In [37]:
class AdministrativeAgent:
    def __init__(self, client):
        self.client = client
        self.bed_status = {}  # Estado de las camas
        self.appointments = {}  # Registro de citas
        self.resources = {}  # Estado de recursos
        self.system_prompt = """
        Eres un administrador hospitalario experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "bed_management": {
                "available_beds": {
                    "general": número,
                    "intensive_care": número,
                    "emergency": número,
                    "isolation": número
                },
                "recommended_unit": "unidad recomendada",
                "estimated_stay": "tiempo estimado",
                "special_requirements": ["requisito 1", "requisito 2", ...]
            },
            "resource_allocation": {
                "staff_needed": [
                    {
                        "role": "tipo de personal",
                        "quantity": número,
                        "urgency": "alta/media/baja"
                    }
                ],
                "equipment_needed": [
                    {
                        "item": "nombre del equipo",
                        "quantity": número,
                        "status": "disponible/en uso/no disponible"
                    }
                ],
                "medications_status": [
                    {
                        "medication": "nombre del medicamento",
                        "stock_level": "alto/medio/bajo",
                        "reorder_needed": boolean
                    }
                ]
            },
            "cost_estimation": {
                "estimated_daily_cost": número,
                "insurance_coverage": "tipo de cobertura",
                "additional_expenses": [
                    {
                        "concept": "descripción",
                        "amount": número
                    }
                ]
            },
            "scheduling": {
                "appointments": [
                    {
                        "type": "tipo de cita",
                        "department": "departamento",
                        "priority": "alta/media/baja",
                        "estimated_duration": "duración estimada"
                    }
                ],
                "follow_up_required": boolean,
                "recommended_timeline": "tiempo recomendado"
            }
        }
        """

    def manage_admission(self, patient_id: str, medical_data: Dict, insurance_info: str = None) -> Dict:
        prompt = f"""
        Gestiona la admisión del siguiente paciente:
        
        ID del paciente: {patient_id}
        Datos médicos:
        {json.dumps(medical_data, indent=2)}
        
        Información del seguro:
        {insurance_info if insurance_info else "Sin información de seguro"}
        
        Proporciona una evaluación administrativa completa.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            admission_plan = json.loads(response.choices[0].message.content)
            self._update_resources(admission_plan)
            return admission_plan
        except Exception as e:
            return {
                "error": "Error en la gestión administrativa",
                "details": str(e)
            }

    def _update_resources(self, admission_plan: Dict) -> None:
        """
        Actualiza el estado de los recursos basado en el plan de admisión
        """
        if "bed_management" in admission_plan:
            unit = admission_plan["bed_management"]["recommended_unit"]
            if unit not in self.bed_status:
                self.bed_status[unit] = {
                    "total": 20,  # Ejemplo de capacidad
                    "occupied": 0
                }
            self.bed_status[unit]["occupied"] += 1

    def get_resource_status(self) -> Dict:
        """
        Obtiene el estado actual de los recursos
        """
        return {
            "bed_status": self.bed_status,
            "appointments": self.appointments,
            "resources": self.resources
        }

    def schedule_appointment(self, patient_id: str, appointment_type: str, urgency: str) -> Dict:
        """
        Programa una cita para el paciente
        """
        prompt = f"""
        Programa una cita para:
        
        ID del paciente: {patient_id}
        Tipo de cita: {appointment_type}
        Nivel de urgencia: {urgency}
        
        Considera la disponibilidad y prioridad.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            scheduling = json.loads(response.choices[0].message.content)
            if patient_id not in self.appointments:
                self.appointments[patient_id] = []
            self.appointments[patient_id].append(scheduling)
            return scheduling
        except Exception as e:
            return {
                "error": "Error en la programación de cita",
                "details": str(e)
            }

In [38]:

# Probemos el AdministrativeAgent
admin = AdministrativeAgent(client)

In [39]:
# Caso de prueba para admisión de paciente cardíaco
medical_data = {
    "diagnosis": "Infarto agudo de miocardio",
    "urgency_level": 1,
    "special_needs": [
        "Monitoreo cardíaco continuo",
        "Acceso a desfibrilador"
    ],
    "estimated_stay": "5-7 días"
}

insurance_info = """
Seguro médico: CompañíaSalud
Tipo de póliza: Premium
Cobertura: 80% de gastos hospitalarios
Deducible: $1,000
"""

In [40]:
# Gestionar la admisión
admission_result = admin.manage_admission("P001", medical_data, insurance_info)
print("Plan de admisión:")
print(json.dumps(admission_result, indent=2, ensure_ascii=False))

Plan de admisión:
{
  "bed_management": {
    "available_beds": {
      "general": 20,
      "intensive_care": 5,
      "emergency": 10,
      "isolation": 3
    },
    "recommended_unit": "Unidad de Cuidados Intensivos",
    "estimated_stay": "5-7 días",
    "special_requirements": [
      "Monitoreo cardíaco continuo",
      "Acceso a desfibrilador"
    ]
  },
  "resource_allocation": {
    "staff_needed": [
      {
        "role": "Cardiólogo",
        "quantity": 2,
        "urgency": "alta"
      },
      {
        "role": "Enfermera especializada",
        "quantity": 3,
        "urgency": "alta"
      }
    ],
    "equipment_needed": [
      {
        "item": "Desfibrilador",
        "quantity": 1,
        "status": "disponible"
      },
      {
        "item": "Monitor cardíaco",
        "quantity": 1,
        "status": "disponible"
      }
    ],
    "medications_status": [
      {
        "medication": "Aspirina",
        "stock_level": "alto",
        "reorder_needed": false

In [41]:
# Programar una cita de seguimiento
appointment = admin.schedule_appointment("P001", "Seguimiento cardiológico", "alta")
print("\nProgramación de cita:")
print(json.dumps(appointment, indent=2, ensure_ascii=False))


Programación de cita:
{
  "bed_management": {
    "available_beds": {
      "general": 10,
      "intensive_care": 5,
      "emergency": 8,
      "isolation": 3
    },
    "recommended_unit": "Unidad de Cardiología",
    "estimated_stay": "2-3 días",
    "special_requirements": [
      "Monitoreo cardíaco",
      "Electrocardiograma"
    ]
  },
  "resource_allocation": {
    "staff_needed": [
      {
        "role": "Cardiólogo",
        "quantity": 1,
        "urgency": "alta"
      },
      {
        "role": "Enfermera especializada",
        "quantity": 2,
        "urgency": "media"
      }
    ],
    "equipment_needed": [
      {
        "item": "Electrocardiógrafo",
        "quantity": 1,
        "status": "disponible"
      },
      {
        "item": "Desfibrilador",
        "quantity": 1,
        "status": "disponible"
      }
    ],
    "medications_status": [
      {
        "medication": "Aspirina",
        "stock_level": "alto",
        "reorder_needed": false
      },
    

In [42]:
# Verificar estado de recursos
resource_status = admin.get_resource_status()
print("\nEstado de recursos:")
print(json.dumps(resource_status, indent=2, ensure_ascii=False))


Estado de recursos:
{
  "bed_status": {
    "Unidad de Cuidados Intensivos": {
      "total": 20,
      "occupied": 1
    }
  },
  "appointments": {
    "P001": [
      {
        "bed_management": {
          "available_beds": {
            "general": 10,
            "intensive_care": 5,
            "emergency": 8,
            "isolation": 3
          },
          "recommended_unit": "Unidad de Cardiología",
          "estimated_stay": "2-3 días",
          "special_requirements": [
            "Monitoreo cardíaco",
            "Electrocardiograma"
          ]
        },
        "resource_allocation": {
          "staff_needed": [
            {
              "role": "Cardiólogo",
              "quantity": 1,
              "urgency": "alta"
            },
            {
              "role": "Enfermera especializada",
              "quantity": 2,
              "urgency": "media"
            }
          ],
          "equipment_needed": [
            {
              "item": "Electrocardió

In [43]:
class EmergencyRoomAgent:
    def __init__(self, client):
        self.client = client
        self.active_emergencies = {}  # Casos activos
        self.ambulance_status = {}    # Estado de ambulancias
        self.system_prompt = """
        Eres un coordinador de sala de emergencias experto. DEBES responder ÚNICAMENTE con un JSON válido que contenga los siguientes campos:
        {
            "emergency_assessment": {
                "case_priority": "rojo/naranja/amarillo/verde/azul",
                "required_resources": [
                    {
                        "type": "tipo de recurso",
                        "quantity": número,
                        "urgency": "inmediata/alta/media/baja"
                    }
                ],
                "estimated_response_time": "tiempo estimado",
                "coordination_needs": ["necesidad 1", "necesidad 2", ...]
            },
            "ambulance_dispatch": {
                "dispatch_required": boolean,
                "type": "básica/avanzada/especializada",
                "eta": "tiempo estimado de llegada",
                "special_equipment": ["equipo 1", "equipo 2", ...],
                "medical_team": ["rol 1", "rol 2", ...]
            },
            "hospital_coordination": {
                "receiving_department": "departamento",
                "required_specialists": ["especialista 1", "especialista 2", ...],
                "isolation_needed": boolean,
                "special_protocols": ["protocolo 1", "protocolo 2", ...]
            },
            "external_coordination": {
                "other_hospitals": [
                    {
                        "name": "nombre del hospital",
                        "distance": "distancia en km",
                        "availability": "disponible/parcial/no disponible",
                        "specialties": ["especialidad 1", "especialidad 2", ...]
                    }
                ],
                "emergency_services": ["servicio 1", "servicio 2", ...],
                "police_required": boolean,
                "fire_department_required": boolean
            },
            "action_plan": {
                "immediate_actions": ["acción 1", "acción 2", ...],
                "contingency_plans": ["plan 1", "plan 2", ...],
                "communication_protocol": "protocolo a seguir",
                "documentation_requirements": ["documento 1", "documento 2", ...]
            }
        }
        """

    def manage_emergency(self, emergency_data: Dict, location: str = None) -> Dict:
        """
        Gestiona un caso de emergencia y coordina la respuesta
        """
        prompt = f"""
        Coordina la siguiente emergencia:
        
        Datos de la emergencia:
        {json.dumps(emergency_data, indent=2)}
        
        Ubicación: {location if location else 'En el hospital'}
        
        Proporciona un plan de acción y coordinación completo.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            emergency_plan = json.loads(response.choices[0].message.content)
            self._update_emergency_status(emergency_data.get("case_id"), emergency_plan)
            return emergency_plan
        except Exception as e:
            return {
                "error": "Error en la gestión de emergencia",
                "details": str(e)
            }

    def dispatch_ambulance(self, case_id: str, location: str, severity: str) -> Dict:
        """
        Gestiona el envío de ambulancias
        """
        prompt = f"""
        Coordina el envío de ambulancia para:
        
        ID del caso: {case_id}
        Ubicación: {location}
        Nivel de severidad: {severity}
        
        Determina el tipo de ambulancia y equipo necesario.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            dispatch_plan = json.loads(response.choices[0].message.content)
            self._update_ambulance_status(case_id, dispatch_plan)
            return dispatch_plan
        except Exception as e:
            return {
                "error": "Error en el despacho de ambulancia",
                "details": str(e)
            }

    def coordinate_mass_casualty(self, incident_data: Dict) -> Dict:
        """
        Coordina la respuesta a incidentes con múltiples víctimas
        """
        prompt = f"""
        Coordina la respuesta a un incidente con múltiples víctimas:
        
        Datos del incidente:
        {json.dumps(incident_data, indent=2)}
        
        Proporciona un plan de coordinación completo.
        """
        
        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.2,
                max_tokens=1024,
                response_format={"type": "json_object"}
            )
            
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            return {
                "error": "Error en la coordinación de incidente masivo",
                "details": str(e)
            }

    def _update_emergency_status(self, case_id: str, emergency_plan: Dict) -> None:
        """
        Actualiza el estado de una emergencia activa
        """
        if case_id:
            self.active_emergencies[case_id] = {
                "timestamp": datetime.now().isoformat(),
                "plan": emergency_plan,
                "status": "active"
            }

    def _update_ambulance_status(self, case_id: str, dispatch_plan: Dict) -> None:
        """
        Actualiza el estado de las ambulancias
        """
        self.ambulance_status[case_id] = {
            "timestamp": datetime.now().isoformat(),
            "dispatch_plan": dispatch_plan,
            "status": "dispatched"
        }

    def get_active_emergencies(self) -> Dict:
        """
        Obtiene todas las emergencias activas
        """
        return self.active_emergencies

In [44]:

# Probemos el EmergencyRoomAgent
emergency = EmergencyRoomAgent(client)

In [45]:

# Caso de prueba: Emergencia cardíaca fuera del hospital
emergency_data = {
    "case_id": "E001",
    "patient_condition": {
        "main_symptom": "Dolor torácico agudo",
        "consciousness": "Consciente",
        "vital_signs": {
            "blood_pressure": "160/95",
            "heart_rate": "120",
            "oxygen_saturation": "92%"
        },
        "risk_factors": [
            "Hipertensión",
            "Fumador activo",
            "Obesidad"
        ]
    },
    "scene_conditions": {
        "accessibility": "Buena",
        "safety_concerns": "Ninguna",
        "distance_to_hospital": "15 km"
    }
}

In [46]:
# Gestionar la emergencia
emergency_response = emergency.manage_emergency(
    emergency_data,
    location="Calle Principal 123, Apartamento 4B"
)
print("Plan de emergencia:")
print(json.dumps(emergency_response, indent=2, ensure_ascii=False))

Plan de emergencia:
{
  "emergency_assessment": {
    "case_priority": "rojo",
    "required_resources": [
      {
        "type": "ambulancia avanzada",
        "quantity": 1,
        "urgency": "inmediata"
      },
      {
        "type": "personal médico especializado",
        "quantity": 2,
        "urgency": "alta"
      }
    ],
    "estimated_response_time": "10 minutos",
    "coordination_needs": [
      "evaluación médica inmediata",
      "monitoreo de signos vitales",
      "administración de medicamentos para el dolor y la presión arterial"
    ]
  },
  "ambulance_dispatch": {
    "dispatch_required": true,
    "type": "avanzada",
    "eta": "8 minutos",
    "special_equipment": [
      "desfibrilador",
      "oxímetro de pulso"
    ],
    "medical_team": [
      "médico de emergencias",
      "enfermera especializada"
    ]
  },
  "hospital_coordination": {
    "receiving_department": "urgencias",
    "required_specialists": [
      "cardiólogo",
      "neumólogo"
    ],


In [47]:
# Despachar ambulancia
ambulance_dispatch = emergency.dispatch_ambulance(
    "E001",
    "Calle Principal 123, Apartamento 4B",
    "alta"
)
print("\nDespacho de ambulancia:")
print(json.dumps(ambulance_dispatch, indent=2, ensure_ascii=False))


Despacho de ambulancia:
{
  "emergency_assessment": {
    "case_priority": "rojo",
    "required_resources": [
      {
        "type": "ambulancia",
        "quantity": 1,
        "urgency": "inmediata"
      },
      {
        "type": "equipo de soporte vital",
        "quantity": 1,
        "urgency": "inmediata"
      }
    ],
    "estimated_response_time": "5 minutos",
    "coordination_needs": [
      "comunicación con el equipo de emergencia",
      "evaluación del paciente"
    ]
  },
  "ambulance_dispatch": {
    "dispatch_required": true,
    "type": "avanzada",
    "eta": "3 minutos",
    "special_equipment": [
      "desfibrilador",
      "equipo de ventilación"
    ],
    "medical_team": [
      "médico",
      "enfermera",
      "técnico de emergencia"
    ]
  },
  "hospital_coordination": {
    "receiving_department": "urgencias",
    "required_specialists": [
      "médico de emergencia",
      "cirujano"
    ],
    "isolation_needed": false,
    "special_protocols": [


In [48]:
# Verificar emergencias activas
active_cases = emergency.get_active_emergencies()
print("\nEmergencias activas:")
print(json.dumps(active_cases, indent=2, ensure_ascii=False))


Emergencias activas:
{
  "E001": {
    "timestamp": "2025-01-12T00:01:14.735518",
    "plan": {
      "emergency_assessment": {
        "case_priority": "rojo",
        "required_resources": [
          {
            "type": "ambulancia avanzada",
            "quantity": 1,
            "urgency": "inmediata"
          },
          {
            "type": "personal médico especializado",
            "quantity": 2,
            "urgency": "alta"
          }
        ],
        "estimated_response_time": "10 minutos",
        "coordination_needs": [
          "evaluación médica inmediata",
          "monitoreo de signos vitales",
          "administración de medicamentos para el dolor y la presión arterial"
        ]
      },
      "ambulance_dispatch": {
        "dispatch_required": true,
        "type": "avanzada",
        "eta": "8 minutos",
        "special_equipment": [
          "desfibrilador",
          "oxímetro de pulso"
        ],
        "medical_team": [
          "médico de eme

In [49]:
def process_complete_hospital_case(
    triage_agent,
    specialist_agent,
    nurse_agent,
    pharmacist_agent,
    admin_agent,
    emergency_agent,
    patient_case: str,
    location: str = None
) -> Dict:
    """
    Procesa un caso hospitalario completo usando todos los agentes disponibles
    """
    # Generamos un ID único para el paciente
    patient_id = f"P{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    # 1. Evaluación inicial de triaje
    triage_result = triage_agent.evaluate_patient(patient_case)
    urgency_level = triage_result.get("urgency_level", 5)
    
    # 2. Si es muy urgente (nivel 1), activamos protocolo de emergencia
    if urgency_level == 1:
        emergency_data = {
            "case_id": patient_id,
            "patient_condition": {
                "main_symptom": patient_case,
                "triage_assessment": triage_result
            }
        }
        emergency_response = emergency_agent.manage_emergency(emergency_data, location)
        
        if location and location != "En el hospital":
            ambulance_dispatch = emergency_agent.dispatch_ambulance(
                patient_id, location, "alta"
            )
    
    # 3. Evaluación del especialista
    department = triage_result["department"]
    specialist_evaluation = specialist_agent.evaluate_case(patient_case, department)
    
    # 4. Revisión farmacéutica
    pharmacy_review = pharmacist_agent.review_prescription(
        patient_id,
        specialist_evaluation.get("prescriptions", []),
        patient_case
    )
    
    # 5. Plan de enfermería
    nursing_care = nurse_agent.monitor_patient(
        patient_id,
        patient_case,
        specialist_evaluation
    )
    
    # 6. Gestión administrativa
    admin_data = {
        "diagnosis": specialist_evaluation.get("diagnosis"),
        "urgency_level": urgency_level,
        "special_needs": nursing_care.get("care_tasks", []),
        "estimated_stay": "1-2 días" if urgency_level > 3 else "3-7 días"
    }
    
    admin_plan = admin_agent.manage_admission(patient_id, admin_data)
    
    # 7. Preparar respuesta completa
    return {
        "case_id": patient_id,
        "status": "EMERGENCY" if urgency_level == 1 else "URGENT" if urgency_level <= 3 else "REGULAR",
        "triage_evaluation": triage_result,
        "emergency_response": emergency_response if urgency_level == 1 else None,
        "specialist_evaluation": specialist_evaluation,
        "pharmacy_review": pharmacy_review,
        "nursing_care": nursing_care,
        "administrative_plan": admin_plan,
        "next_steps": triage_result.get("immediate_actions", []),
        "follow_up": specialist_evaluation.get("follow_up"),
        "timestamp": datetime.now().isoformat()
    }

In [50]:
# Probemos la función completa
test_case = """
Paciente de 58 años presenta:
- Dolor intenso en el pecho que irradia al brazo izquierdo
- Sudoración fría
- Dificultad para respirar
- Náuseas
- Antecedentes de hipertensión y diabetes
- Fumador activo
Tiempo de evolución: 30 minutos
"""

In [51]:
complete_case = process_complete_hospital_case(
    triage_agent,
    specialist,
    nurse,
    pharmacist,
    admin,
    emergency,
    test_case,
    location="Calle Principal 123, Apartamento 4B"
)

print("Procesamiento completo del caso:")
print(json.dumps(complete_case, indent=2, ensure_ascii=False))

Procesamiento completo del caso:
{
  "case_id": "P20250112000540",
  "status": "EMERGENCY",
  "triage_evaluation": {
    "urgency_level": 1,
    "department": "Cardiología",
    "immediate_actions": [
      "Llamar a emergencias",
      "Administrar oxígeno",
      "Monitorear signos vitales",
      "Preparar para posible reanimación cardiopulmonar"
    ],
    "reasoning": "Los síntomas presentados sugieren un posible infarto de miocardio, considerando el dolor intenso en el pecho que irradia al brazo izquierdo, la sudoración fría, la dificultad para respirar y las náuseas. Los antecedentes de hipertensión, diabetes y el hábito de fumar aumentan el riesgo cardiovascular. La evolución rápida de los síntomas en 30 minutos indica la necesidad de atención inmediata."
  },
  "emergency_response": {
    "emergency_assessment": {
      "case_priority": "rojo",
      "required_resources": [
        {
          "type": "ambulancia avanzada",
          "quantity": 1,
          "urgency": "inmedi